# 15、载入预训练词向量-CNN文本分类
Glove词向量的下载地址：http://nlp.stanford.edu/data/glove.6B.zip,(source page: http://nlp.stanford.edu/projects/glove/),20个文本分类数据集：http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.html


In [1]:
from __future__ import print_function

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.layers import Dense, Dropout, Activation
from keras.models import Model

Using TensorFlow backend.


In [2]:
BASE_DIR = '/home/yude/Public/down'
GLOVE_DIR = BASE_DIR + '/glove.6B/'
TEXT_DATA_DIR = BASE_DIR + '/20_newsgroup/'
MAX_SEQUENCE_LENGTH = 1000 # 句子最长1000
MAX_NB_WORDS = 20000   # 取最常用20000个词
EMBEDDING_DIM = 100    # 词向量维数100
VALIDATION_SPLIT = 0.2  

In [3]:
print('Indexing word vectors.')

embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()   # 按空格进行分隔，并组成list
    word = values[0]        # 一行中第一是单词，后面是词向量
    coefs = np.asarray(values[1:], dtype='float32')  # 转类型
    embeddings_index[word] = coefs   # 组成字典
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400000 word vectors.


In [4]:
embeddings_index['the']

array([-0.038194  , -0.24487001,  0.72812003, -0.39961001,  0.083172  ,
        0.043953  , -0.39140999,  0.3344    , -0.57545   ,  0.087459  ,
        0.28786999, -0.06731   ,  0.30906001, -0.26383999, -0.13231   ,
       -0.20757   ,  0.33395001, -0.33848   , -0.31742999, -0.48335999,
        0.1464    , -0.37303999,  0.34577   ,  0.052041  ,  0.44946   ,
       -0.46970999,  0.02628   , -0.54154998, -0.15518001, -0.14106999,
       -0.039722  ,  0.28277001,  0.14393   ,  0.23464   , -0.31020999,
        0.086173  ,  0.20397   ,  0.52623999,  0.17163999, -0.082378  ,
       -0.71787   , -0.41531   ,  0.20334999, -0.12763   ,  0.41367   ,
        0.55186999,  0.57907999, -0.33476999, -0.36559001, -0.54856998,
       -0.062892  ,  0.26583999,  0.30204999,  0.99774998, -0.80480999,
       -3.0243001 ,  0.01254   , -0.36941999,  2.21670008,  0.72201002,
       -0.24978   ,  0.92136002,  0.034514  ,  0.46744999,  1.10790002,
       -0.19358   , -0.074575  ,  0.23353   , -0.052062  , -0.22

In [5]:
# second, prepare text samples and their labels
print('Processing text dataset')
texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids
for name in sorted(os.listdir(TEXT_DATA_DIR)):   # 对20个文件按字母表排序，依次处理
    path = os.path.join(TEXT_DATA_DIR, name)     # 为每一个文件指定具体文件路径
    if os.path.isdir(path): 
        label_id = len(labels_index)
        labels_index[name] = label_id
        for fname in sorted(os.listdir(path)):
            if fname.isdigit():
                fpath = os.path.join(path, fname)
                if sys.version_info < (3,):
                    f = open(fpath)
                else:
                    f = open(fpath, encoding='latin-1')
                t = f.read()
                i = t.find('\n\n')  # skip header
                if 0 < i:
                    t = t[i:]
                texts.append(t)
                f.close()
                labels.append(label_id)

print('Found %s texts.' % len(texts))

# 上面处理把文章的抬头部分去掉,texts为每篇文章内容，labels是对应文章的标签

Processing text dataset
Found 19997 texts.


In [6]:
# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)  # 最常用的20000个单词
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)   # 把文章中的词转为数字序列list
word_index = tokenizer.word_index                 # 把文章中的单词与数字号做成字典
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Found 174105 unique tokens.
Shape of data tensor: (19997, 1000)
Shape of label tensor: (19997, 20)


In [7]:
len(word_index)

174105

In [8]:
# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

In [9]:
print('Preparing embedding matrix.')

# prepare embedding matrix
num_words = min(MAX_NB_WORDS, len(word_index))  # 20000
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():     # 遍历字典中每一个词，
    if i >= MAX_NB_WORDS:              # 不在这20000个词内，跳过
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:           # 如果这20000个词可以在词向理中找到
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector    # 现在的词向量，是数字号与向量，而不是原来的单词与向量的结合

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],     # 这个参数我并没有在文档中看到，
                            input_length=MAX_SEQUENCE_LENGTH,   # 与序列长度一样
                            trainable=False)    # 不再做训练

Preparing embedding matrix.


In [10]:
embedding_matrix[0]  # 第一个为0向量，即不在20000个词内的向量为0向量

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [11]:
embedding_matrix[3]

array([-0.1529    , -0.24279   ,  0.89837003,  0.16996001,  0.53516001,
        0.48784   , -0.58825999, -0.17982   , -1.35810006,  0.42541   ,
        0.15377   ,  0.24214999,  0.13474   ,  0.41192999,  0.67043   ,
       -0.56418002,  0.42985001, -0.012183  , -0.11677   ,  0.31781   ,
        0.054177  , -0.054273  ,  0.35516   , -0.30241001,  0.31434   ,
       -0.33846   ,  0.71714997, -0.26855001, -0.15837   , -0.47466999,
        0.051581  , -0.33252001,  0.15003   , -0.12989999, -0.54617   ,
       -0.37843001,  0.64261001,  0.82187003, -0.080006  ,  0.078479  ,
       -0.96976   , -0.57740998,  0.56490999, -0.39873001, -0.057099  ,
        0.19743   ,  0.065706  , -0.48091999, -0.20125   , -0.40834001,
        0.39456001, -0.02642   , -0.11838   ,  1.01199996, -0.53171003,
       -2.74740005, -0.042981  , -0.74848998,  1.75740004,  0.59085   ,
        0.04885   ,  0.78267002,  0.38497001,  0.42096999,  0.67882001,
        0.10337   ,  0.63279998, -0.026595  ,  0.58647001, -0.44

In [12]:
print('Training model.')

# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Dropout(0.5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Dropout(0.5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(35)(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(len(labels_index), activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
model.summary()

Training model.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 100)         2000000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 996, 128)          64128     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 199, 128)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 199, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 195, 128)          82048     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 39, 128)           0    

In [13]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=15,
          verbose=2,
          validation_data=(x_val, y_val))

Train on 15998 samples, validate on 3999 samples
Epoch 1/15
16s - loss: 2.6620 - acc: 0.1391 - val_loss: 2.2572 - val_acc: 0.2663
Epoch 2/15
15s - loss: 1.9020 - acc: 0.3300 - val_loss: 1.6002 - val_acc: 0.4514
Epoch 3/15
15s - loss: 1.5363 - acc: 0.4539 - val_loss: 1.3400 - val_acc: 0.5509
Epoch 4/15
15s - loss: 1.3513 - acc: 0.5304 - val_loss: 1.2143 - val_acc: 0.6119
Epoch 5/15
15s - loss: 1.2098 - acc: 0.5791 - val_loss: 1.0692 - val_acc: 0.6597
Epoch 6/15
15s - loss: 1.1180 - acc: 0.6115 - val_loss: 1.0217 - val_acc: 0.6637
Epoch 7/15
15s - loss: 1.0459 - acc: 0.6349 - val_loss: 0.9936 - val_acc: 0.6727
Epoch 8/15
16s - loss: 0.9843 - acc: 0.6642 - val_loss: 0.9303 - val_acc: 0.7057
Epoch 9/15
16s - loss: 0.9285 - acc: 0.6819 - val_loss: 0.9001 - val_acc: 0.7144
Epoch 10/15
16s - loss: 0.8871 - acc: 0.6965 - val_loss: 0.8476 - val_acc: 0.7297
Epoch 11/15
16s - loss: 0.8508 - acc: 0.7158 - val_loss: 0.8654 - val_acc: 0.7324
Epoch 12/15
16s - loss: 0.8124 - acc: 0.7236 - val_loss: 0